# Phase 0: Hello BigQuery AI
This notebook demonstrates minimal embedding + vector search + optional text generation using BigQuery AI.


In [ ]:
# Imports & environment configuration
import os
from pathlib import Path

PROJECT_ID = os.getenv('BQ_PROJECT_ID', 'bq_project_northstar')
DATASET = os.getenv('BQ_DATASET', 'demo_ai')
LOCATION = os.getenv('BQ_LOCATION', 'US')
EMBED_MODEL = os.getenv('BQ_EMBED_MODEL', 'text-embedding-004')
GEN_MODEL = os.getenv('BQ_GEN_MODEL', 'gemini-1.5-pro')
TOP_K = 5
print('Config:', PROJECT_ID, DATASET, LOCATION, EMBED_MODEL, GEN_MODEL)


## (Optional) Authentication
If running on Kaggle with BigQuery integration, auth is implicit. Else uncomment the service account block below and provide a JSON file path in env `BQ_SERVICE_ACCOUNT_JSON`.


In [ ]:
# Optional service account auth (commented)
# from google.cloud import bigquery
# import json, os
# sa_json = os.getenv('BQ_SERVICE_ACCOUNT_JSON')
# if sa_json and Path(sa_json).exists():
#     os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = sa_json
# client = bigquery.Client(project=PROJECT_ID, location=LOCATION)


In [ ]:
# Create dataset if not exists and base table with sample texts
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location=LOCATION)
client.create_dataset(f'{PROJECT_ID}.{DATASET}', exists_ok=True)
rows = [
    (1, 'Login page returns 500 after password reset.'),
    (2, 'Image upload fails: content-type mismatch.'),
    (3, 'DB connection timeout in staging during deploy.'),
    (4, 'Checkout error: stripe webhook signature invalid.'),
    (5, 'Search results empty when filters combined.'),
    (6, 'Mobile app crash on Android 14, camera permission.'),
    (7, 'TLS certificate expired on edge gateway.'),
    (8, 'Cron job skipped due to timezone change.'),
]

schema = [bigquery.SchemaField('id','INT64'), bigquery.SchemaField('text','STRING')]
table_id = f'{PROJECT_ID}.{DATASET}.demo_texts'
try:
    table = client.create_table(bigquery.Table(table_id, schema=schema))
except Exception:
    table = client.get_table(table_id)
if client.get_table(table_id).num_rows == 0:
    errors = client.insert_rows(table, rows)
    assert not errors, errors
print('Base table ready; rows:', client.get_table(table_id).num_rows)


In [ ]:
# Run embeddings SQL to materialize demo_texts_emb
from pathlib import Path
EMBED_SQL_PATH = Path('sql/embeddings.sql')
sql_src = EMBED_SQL_PATH.read_text()
rendered = (sql_src
    .replace('${PROJECT_ID}', PROJECT_ID)
    .replace('${DATASET}', DATASET)
    .replace('${EMBED_MODEL}', EMBED_MODEL)
    .replace('${SOURCE_TABLE}', 'demo_texts')
)
job = client.query(rendered)
job.result()
print('Embeddings table created.')


In [ ]:
# Inspect embedded table
e_emb = client.get_table(f'{PROJECT_ID}.{DATASET}.demo_texts_emb')
print('Embedded rows:', e_emb.num_rows)
preview = client.query(
    f'SELECT id, ARRAY_LENGTH(embedding) AS dim FROM `{PROJECT_ID}.{DATASET}.demo_texts_emb` LIMIT 3'
).to_dataframe()
preview


In [ ]:
# Vector search for a sample query
VS_SQL_PATH = Path('sql/vector_search.sql')
vs_src = VS_SQL_PATH.read_text()
vs_rendered = (vs_src
    .replace('${PROJECT_ID}', PROJECT_ID)
    .replace('${DATASET}', DATASET)
    .replace('${EMBED_MODEL}', EMBED_MODEL)
    .replace('${QUERY_TEXT}', "'login error after reset'")
    .replace('${TOP_K}', str(TOP_K))
)
vs_df = client.query(vs_rendered).to_dataframe()
vs_df


In [ ]:
# Optional text generation classification
GEN_SQL_PATH = Path('sql/generate_text.sql')
gen_src = GEN_SQL_PATH.read_text()
gen_rendered = (gen_src
    .replace('${PROJECT_ID}', PROJECT_ID)
    .replace('${DATASET}', DATASET)
    .replace('${GEN_MODEL}', GEN_MODEL)
    .replace('${LIMIT}', '3')
)
gen_df = client.query(gen_rendered).to_dataframe()
gen_df


In [ ]:
# Run verifier to confirm required ML constructs
from src import verifier as v_mod
ok = v_mod.verify_sql_directory(Path('sql'))
print('Verifier result:', ok)


## Summary
We created a demo dataset, generated embeddings, performed vector search, and optionally classified rows with text generation. Next: vector index + triage loop (Phase 1).


In [ ]:
# 4. SQL Loader & Execution Helpers

def load_sql(name: str) -> str:
    path = SQL_DIR / name
    return path.read_text(encoding="utf-8")


def render_sql(template: str, params: dict[str, Any]) -> str:
    sql = template
    for k, v in params.items():
        sql = sql.replace(f"${{${k}}}", v)
    return sql


def run_query(sql: str):
    if not bigquery:
        print("(skip) bigquery lib not available")
        return []
    job = client.query(sql)
    return list(job)


def dry_run(sql: str) -> int:
    if not bigquery:
        return 0
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    job = client.query(sql, job_config=job_config)
    return job.total_bytes_processed  # type: ignore[attr-defined]

print("Helpers ready.")


In [ ]:
# 5. Inspect Source Data (placeholder)
print("TODO: Preview source tables once dataset is populated.")

# 6. Generate Embeddings (ML.GENERATE_EMBEDDING) (placeholder)
print("TODO: Execute embeddings.sql with parameters.")

# 7. Persist Embeddings Table (placeholder)
print("TODO: CREATE OR REPLACE target embeddings table.")

# 8. Create / Use Vector Index (placeholder)
print("TODO: Demonstrate index creation or document exact search.")

# 9. Vector Search (VECTOR_SEARCH) (placeholder)
print("TODO: Run vector_search.sql and display results.")

# 10. End-to-End Retrieval Pipeline (GraphRAG) (placeholder)
print("TODO: Implement multi-hop retrieval in retrieval module.")

# 11. Augmented Generation Demo (placeholder)
print("TODO: Compose context + call model or placeholder generator.")

# 12. Evaluation & Verification Checks (placeholder)
print("TODO: Execute verifier functions and summarize.")

# 13. CLI Integration Example (placeholder)
print("TODO: Invoke CLI retrieval command via subprocess.")

# 14. Resource & Cost Monitoring (placeholder)
print("TODO: Track dry run bytes and actual bytes in a dataframe.")

# 15. Cleanup Temporary Assets (placeholder)
print("TODO: Drop temp tables / datasets if created.")
